#Téléchargement des packages

In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import math

nltk.download('punkt_tab')
# 1️⃣ Télécharger les ressources nécessaires
nltk.download('punkt')       # tokenizer
nltk.download('stopwords')   # stopwords français
# Télécharger les ressources nécessaires (la première fois)
nltk.download('punkt')
nltk.download('stopwords')



[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#On enlève les stopwords

In [2]:
# Liste des stopwords en français
#on va travailler sur ces deux textes simples
doc1 = "le domaine de l'IA est un domaine modifié en croissance "
doc2 = "le domaine de l'it va être modifier par l'IA domaine modifié"
stop_words = set(stopwords.words('french'))

def remove_stopwords(text):
    words = word_tokenize(text, language='french')
    filtered_words = [word for word in words if word.lower() not in stop_words and word.isalpha()]
    return " ".join(filtered_words)

# Appliquer la fonction aux deux textes
texte1_sans_stopwords = remove_stopwords(doc1)
texte2_sans_stopwords = remove_stopwords(doc2)



corpus=doc1+"vSEPARATION "+doc2 #pour ne pas prendre en compte le faite que le dernier mots du premier doc est lié aux premier mot du deuxième doc
corpus=corpus.lower().split()

#Extraction du vocabulaire

In [3]:
# Tokenisation
tokens1 = texte1_sans_stopwords.lower().split()
tokens2 = texte2_sans_stopwords.lower().split()

# Comptage des mots
count1 = Counter(tokens1)
count2 = Counter(tokens2)

#on enlève les redondances


unique_tokens1= []
for token in tokens1:
    if token not in unique_tokens1:
        unique_tokens1.append(token)

unique_tokens2= []
for token in tokens2:
    if token not in unique_tokens2:
        unique_tokens2.append(token)



In [4]:
unique_tokens1, unique_tokens2

(['domaine', 'modifié', 'croissance'],
 ['domaine', 'va', 'être', 'modifier', 'modifié'])

In [5]:
corpus

['le',
 'domaine',
 'de',
 "l'ia",
 'est',
 'un',
 'domaine',
 'modifié',
 'en',
 'croissance',
 'vseparation',
 'le',
 'domaine',
 'de',
 "l'it",
 'va',
 'être',
 'modifier',
 'par',
 "l'ia",
 'domaine',
 'modifié']

#Calcul du PMI pour chaque couple de  mots  issu chacun d'un documents

###le résultat sera enregister dans une matrice  



In [6]:
import pandas as pd


# Création d'un DataFrame vide avec colonnes les token du doc1 et index les token du doc2
df = pd.DataFrame(0, index=unique_tokens2, columns=unique_tokens1)

print(df)


          domaine  modifié  croissance
domaine         0        0           0
va              0        0           0
être            0        0           0
modifier        0        0           0
modifié         0        0           0


In [7]:
corpus.count("domaine")

4

In [12]:
#une fonction  qui permet de calculer le nombre de fois que deux mots  se succèdent dans le corpus

def compter_succession(corpus, x, y):

    compteur = 0
    for i in range(len(corpus) - 1):
        if corpus[i] == x and corpus[i + 1] == y:
            compteur += 1
    return compteur

#la fonction qui va calculer les pmi de deux mots

def pmi(mot1,mot2,corpus):
  p_mot1=corpus.count(mot1)/len(corpus)
  p_mot2=corpus.count(mot2)/len(corpus)
  p_mot1_mot2=compter_succession(corpus,mot1,mot2)/len(corpus)
  if p_mot1_mot2==0:
    return 0
  if p_mot1==0 or p_mot2==0:
    return 0
  pmi_value=math.log(p_mot1_mot2/(p_mot2*p_mot1))

  return pmi_value


In [13]:
print(pmi("domaine","modifié",corpus))

1.7047480922384253


In [14]:
#remplissage de la matrice
for row_index, row in df.iterrows():
    for col_name in df.columns:
        if row_index == col_name:
            df.at[row_index, col_name] = 0
        else:
            df.at[row_index, col_name] = pmi(row_index, col_name, corpus)


In [16]:
df

,domaine,modifié,croissance
domaine,0,1.704748,0
va,0,0.000000,0
être,0,0.000000,0
modifier,0,0.000000,0
modifié,0,0.000000,0


#On calcule la moyennes du PMI de tous les mots

In [15]:
#on fait la moyenne

df.values.mean()

np.float64(0.11364987281589502)